In [48]:
import numpy as np
import plotly.graph_objects as go

In [49]:


# Interpolate points from 0,0,0 to 1,1,1
x,y,z = np.linspace(0,1,10), np.linspace(0,1,10), np.linspace(0,1,10)


# Create figure
fig = go.Figure(
    data=[go.Scatter3d(x=[], y=[], z=[],
                     mode="markers",marker=dict(color="red", size=10))])
    
fig.update_layout(
        
         scene = dict(
        
        xaxis=dict(range=[min(x), max(x)], autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False),
        )),


frames = [go.Frame(data= [go.Scatter3d(
                                       x=x[[k]], 
                                       y=y[[k]],
                                       z=z[[k]])],
                   
                   traces= [0],
                   name=f'frame{k}'      
                  )for k  in  range(len(x))]
fig.update(frames=frames),




fig.update_layout(updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))      ])])])


fig.show()

In [64]:
def plot_std(model):
    """Plot the standard deviation of the Gaussian Process for a grid of points between 0 and 1."""
    import numpy as np
    import plotly.graph_objects as go
    import pandas as pd

    n = 100
    feat_grid = np.linspace(0, 1, n)
    X, Y = np.meshgrid(feat_grid, feat_grid)
    Z = np.zeros((n, n))

    steps = 3
    slider_steps = np.linspace(0, 1, steps)

    df = pd.DataFrame()
    for s in slider_steps:
        for i in range(n):
            for j in range(n):
                y_hat, y_std = model.predict(np.array([[X[i, j], Y[i, j], s]]), return_std=True)
                y_hat0, y_hat1 = y_hat[0]
                std0, std1 = y_std[0]
                #Z[i, j] = std0
                df = df.append({'x': X[i, j], 'y': Y[i, j], 'step': s, 'y_hat0':y_hat0, 'std0': std0, 'y_hat1': y_hat1, 'std1': std1}, ignore_index=True)

    return df

    # plot_type = 'surface' # surface or contour
    # if plot_type == 'surface':
    #     fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y)])

    #     # Add training points as scatter plot
    #     x_train = self.x_train[:, 0]
    #     y_train = self.x_train[:, 1]
    #     dummy_train = self.x_train.copy()
    #     dummy_train[:, 2] = 0
    #     _, y_std = self.model.predict(dummy_train, return_std=True)
    #     z_train = y_std[:, 0]
    #     fig.add_trace(go.Scatter3d(x=x_train, y=y_train, z=z_train, mode='markers', marker=dict(color='green', size=3),
    #                         name='Training Points', showlegend=True))

    #     fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Standard Deviation'))
    # elif plot_type == 'contour':
    #     fig = go.Figure(data=go.Contour(z=Z, x=x, y=y, contours=dict(coloring='heatmap')))
    #     fig.update_layout(coloraxis=dict(colorscale='Viridis'))

    # fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    # fig.update_layout(height=800, margin=dict(l=0, r=0, b=0, t=0))
    # fig.show()

    # print("Done plotting.")
    

In [106]:
import numpy as np
import pandas as pd

def plot_std(model):
    """Plot the standard deviation of the Gaussian Process for a grid of points between 0 and 1."""
    n = 11
    feat_grid = np.linspace(0, 1, n)
    X, Y = np.meshgrid(feat_grid, feat_grid)

    steps = 11
    slider_steps = np.linspace(0, 1, steps)

    data = []
    for s in slider_steps:
        y_hats, y_stds = model.predict(np.c_[X.ravel(), Y.ravel(), np.full(X.size, s)], return_std=True)
        y_hats = y_hats.reshape(X.shape[0], X.shape[1], -1)
        y_stds = y_stds.reshape(X.shape[0], X.shape[1], -1)

        for i in range(n):
            for j in range(n):
                y_hat0, y_hat1 = y_hats[i, j]
                std0, std1 = y_stds[i, j]
                data.append({'x': X[i, j], 'y': Y[i, j], 'step': s, 'y_hat0': y_hat0, 'std0': std0, 'y_hat1': y_hat1, 'std1': std1})

    df = pd.DataFrame(data)
    return df

In [107]:
df = plot_std(model)
df

,x,y,step,y_hat0,std0,y_hat1,std1
0,0.0,0.0,0.0,0.500000,0.996086,1.000000,0.996086
1,0.1,0.0,0.0,0.506009,0.995991,1.012018,0.995991
2,0.2,0.0,0.0,0.511509,0.995903,1.023018,0.995903
3,0.3,0.0,0.0,0.516462,0.995824,1.032924,0.995824
4,0.4,0.0,0.0,0.520833,0.995752,1.041667,0.995752
...,...,...,...,...,...,...,...
1326,0.6,1.0,1.0,1.117318,0.980300,2.234637,0.980300
1327,0.7,1.0,1.0,1.128350,0.979905,2.256700,0.979905
1328,0.8,1.0,1.0,1.136364,0.979615,2.272727,0.979615
1329,0.9,1.0,1.0,1.141227,0.979439,2.282454,0.979439


In [103]:
import plotly.express as px

#df = px.data.gapminder()

fig = go.Figure(data=[
    go.Surface(z=df['y_hat0'].values),
    #go.Surface(z=z2, showscale=False, opacity=0.9),
    #go.Surface(z=z3, showscale=False, opacity=0.9)

])

# fig = px.surface_3d(df, x="x", y='y', z="y_hat0",
#                  animation_frame="step",
#             #animation_group="country",
#            #size="pop",
#            #color="continent",
#            #hover_name="country",
#            #log_x=True,
#            #size_max=1,
#            #size=[0.1]*300,
#            range_x=[-0.1,1.1],
#            range_y=[-0.1,1.1],
#            range_z=[0,2]
#            )

#fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.update_layout(height=1000, margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [117]:
import plotly.graph_objects as go
from plotly.graph_objs import *
import numpy as np

n = 100
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
z = np.sin(x ** 2 + y ** 2)
#data =  np.random.rand(10, 10, 10)

steps = 10
fr = np.arange(steps)

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    sliders=[{
        "active": 0,
        "steps": [{"args": [[f], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}],
                   "label": f,
                   "method": "animate"} for f in fr.tolist()],
        "currentvalue": {"visible": True, "prefix": "Frame: "},
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "x": 0,
        "y": 0,
        "len": 1
                }]
)

fig = go.Figure(layout=layout)

fig = go.Figure(layout=layout)

frames = []

X, Y = np.meshgrid(x, y)

for i in range(steps):
    z = np.sin(X*6*np.pi + i/3) + np.cos(Y*6*np.pi + i/3)
    
    trace = go.Surface(
        z=z,
        x=X,
        y=Y,
        opacity=1,
        colorscale="Viridis",
        colorbar=dict(title="Counts"),
        cmin=0,
        cmax=1
    )
    
    frame = go.Frame(data=[trace], layout=go.Layout(title=f"Frame: {fr[i]}"))
    frames.append(frame)
    
    #fig.add_trace(trace)
    #fig.frames = frames

fig.add_trace(frames[0]['data'][0])
fig.frames = frames


fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    margin=dict(l=65, r=50, b=65, t=90)
)

zoom = 1.35

fig.update_layout(
    scene={
        "xaxis": {"nticks": 20},
        "zaxis": {"nticks": 4},
        "camera_eye": {"x": 0.1, "y": 0.4, "z": 0.25},
        "aspectratio": {"x": 0.4 * zoom, "y": 0.4 * zoom, "z": 0.25 * zoom}
    }
)

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
)


fig.show()


In [83]:
df.groupby('step').describe()

x                                                       y       ...  \
      count mean      std  min       25%  50%       75%  max  count mean  ...   
step                                                                      ...   
0.0   100.0  0.5  0.32075  0.0  0.222222  0.5  0.777778  1.0  100.0  0.5  ...   
0.5   100.0  0.5  0.32075  0.0  0.222222  0.5  0.777778  1.0  100.0  0.5  ...   
1.0   100.0  0.5  0.32075  0.0  0.222222  0.5  0.777778  1.0  100.0  0.5  ...   

        y_hat1             std1                                          \
           75%       max  count      mean       std       min       25%   
step                                                                      
0.0   1.246454  1.333333  100.0  0.994572  0.000801  0.993031  0.993913   
0.5   1.586292  1.729730  100.0  0.991412  0.001587  0.988244  0.990122   
1.0   2.041750  2.285714  100.0  0.986153  0.003236  0.979379  0.983581   

                                    
           50%       75%       max  
step                                
0.0   0.994573  0.995277  0.996086  
0.5   0.991457  0.992814  0.994288  
1.0   0.986363  0.989016  0.991701  

[3 rows x 48 columns]